In [3]:
import importlib
import verse
import corpus
import phonetic
import generate_sample
import numpy as np

In [150]:
importlib.reload(phonetic)

dictionary = phonetic.PhoneticDictionary()

song_files = ["lyrics/kendrick_lamar/GoodKid.txt",
             "lyrics/cardi-b/BodakYellow.txt",
             "lyrics/nicki_minaj/ChunLi.txt",
             "lyrics/drake/NiceForWhat.txt",
             "lyrics/hamilton/processed/WaitForIt.txt"]

unknowns = ["logios_output/kendrick_lamar/GoodKid_logios.txt",
           "logios_output/cardi-b/BodakYellow_logios.txt",
           "logios_output/nicki_minaj/ChunLi_logios.txt",
           "logios_output/drake/NiceForWhat_logios.txt",
           "logios_output/hamilton/WaitForIt_logios.txt"]

kendrick_lamar_verse = "Mass hallucination baby \
Ill education baby \
Want to reconnect with your relations \
This is your station baby \
Look inside these walls and you see them having withdrawals \
Of a prisoner on his way \
Trapped inside your desire \
To fire bullets that stray \
Track a tire just tell you I'm tired and ran away \
I should ask a choir what do you require \
To sing a song that acquire me to have faith \
As the record spin I should pray \
For the record I recognize that I'm easily prey \
I got ate alive yesterday \
I got animosity building \
It's probably big as a building"

cardi_b_verse = "I'm the hottest in the street, know you prolly heard of me. \
Got a bag and fixed my teeth, hope you hoes know it ain't cheap. \
And I pay my mama bills, I ain't got no time to chill. \
Think these hoes be mad at me, their baby father run a bill. \
Said, Lil bitch, you can't fuck with me if you wanted to. \
These expensive, these is red bottoms, these is bloody shoes. \
Hit the store, I can get 'em both, I don't wanna choose. \
And I'm quick, cut a nigga off, so don't get comfortable, look."

nicki_minaj_verse = "I went and copped the chopsticks \
Put it in my bun just to pop shit \
I'm always in the top shit \
Box seats, bitch, fuck the gossip \
How many a' them could of did it with finesse? \
Now everybody like, She really is the best. \
You play checkers, couldn't beat me playing chess \
Now I'm about to turn around and beat my chest \
Bitch, it's King Kong, yes, it's King Kong \
Bitch, it's King Kong, this is King Kong \
Chinese ink on, Siamese links on \
Call me 2 Chains, name go ding dong"

drake_verse = "Everybody get your mothafuckin' roll on \
I know shorty and she doesn't want no slow song \
Had a man last year, life goes on \
Haven't let that thing loose, girl, in so long \
You been inside, know you like to lay low \
I've been peepin' what you bringin' to the table \
Workin' hard, girl, everything paid for \
First, last phone bill, car note, cable \
With your phone out, gotta hit them angles \
With your phone out, snappin' like you Fabo \
And you showin' off, but it's alright \
And you showin' off, but it's alright \
It's a short life, yeah"

hamilton_verse = "Love doesn't discriminate, \
Between the sinners, \
And the saints, \
It takes and it takes and it takes, \
And we keep loving anyway, \
We laugh and we cry, \
And we break, \
And we make our mistakes, \
And if there's a reason I'm by her side, \
When so many have tried, \
Then I'm willing to wait for it, \
I'm willing to wait for it"

verses = [kendrick_lamar_verse,
         cardi_b_verse,
         nicki_minaj_verse,
         drake_verse,
         hamilton_verse]

In [5]:
def make_template(text):
    template = verse.VerseTemplate(text, dictionary, breakrules="phrase")
    template.get_rhyme()
    return template

In [75]:
def compute_similarity_score(res1, res2):
    
    print("COMPARING:")
    print("RES1: ", res1)
    print("REST2: ", res2)
    
    # iterate by syllable
    # check word at each syllable position and compare between strings
    
    # both templates should have the same number of syllables
    first_template, second_template = res1[1], res2[1]
    num_syls = first_template.num_syllables
    assert num_syls == second_template.num_syllables
    
    total_matches = 0
    """
    for syl_pos in range(num_syls):
        word1 = first_template.wordList[first_template.matrix_indices[syl_pos][0]]
        word2 = second_template.wordList[second_template.matrix_indices[syl_pos][0]]
        # print(word1, word2)
        if word1.stringRepr == word2.stringRepr:
            print("MATCH: ", word1, word2)
            total_matches += 1
        else:
            print("NO MATCH: ", word1, word2)
    """
    print(total_matches, num_syls)
    return total_matches / num_syls

In [116]:
def intra_similarity(template_text, corpus, unknowns, n_iters):
    
    results = []
    
    # print("Template text: ", template_text)
    
    for i in range(n_iters):
        template = verse.VerseTemplate(template_text, dictionary,
                              breakrules='phrase')
        template.add_unknowns(unknowns)
        template.get_rhyme()
        result_text = generate_sample.fill_template(corp, template, verbose=False)
        # for k, v in sorted(template.verse.items()):
            # print(k, v)
        # break
        # print("\n", result_text)
        results.append((result_text, template))
        
    score_matrix = np.zeros((len(results), len(results)))
    total_lengths = []
    for i in range(len(results)):
        for j in range(len(results)):
            first_result_template = results[i][1]
            second_result_template = results[j][1]
            
            first_result_words = [val[0].stringRepr for _, val in sorted(first_result_template.verse.items())]
            second_result_words = [val[0].stringRepr for _, val in sorted(second_result_template.verse.items())]
            
            total_matches = 0
            total_words = max(len(first_result_words), len(second_result_words))
            for k in range(min(len(first_result_words), len(second_result_words))):
                if first_result_words[k] == second_result_words[k]:
                    total_matches += 1
            total_lengths.append(total_words)        
            score_matrix[i,j] = total_matches / total_words
            # assert(len(first_result_words) == len(second_result_words))
            # print(sorted(first_result_template.verse.items()))
            # print(first_result_words)
            # score_matrix[i,j] = compute_similarity_score(results[i], results[j])
            
    # zero out the diagonal
    np.fill_diagonal(score_matrix, 0)
            
    average_sim_score = np.mean(score_matrix)
    max_sim_score = np.max(score_matrix)
    average_verse_length = np.mean(total_lengths)
    
    # print("Avg similarity score: ", average_sim_score)
    # print("Max similarity score: ", max_sim_score)
    # print(score_matrix)
    
    return average_sim_score, max_sim_score, average_verse_length

In [118]:
importlib.reload(corpus)
importlib.reload(verse)
importlib.reload(generate_sample)

# corp = corpus.WordCorpus(dictionary)

for idx, song in enumerate(song_files):
    
    # idx += 4
    # print("unknowns: ", unknowns[idx])
    print("\n\n\nTesting on ", song)
    corp = corpus.WordCorpus(dictionary)
    corp.initialize(song)
    corp.add_unknowns(unknowns[idx])
    corp.initializeMatrix()
    corp.initializeSylDict()
    
    average_sim_scores = []
    max_sim_scores = []
    average_verse_lengths = []

    for i in range(100):
        avg_sim_score, max_sim_score, avg_v_length = intra_similarity(verses[idx], corp, unknowns[idx], 10)
        average_sim_scores.append(avg_sim_score)
        max_sim_scores.append(max_sim_score)
        average_verse_lengths.append(avg_v_length)
        
        if i % 20 == 0:
            print("Iteration: ", i + 1)
            print("Last avg sim score: ", average_sim_scores[-1])
            print("Last max sim score: ", max_sim_scores[-1])
            print("Last avg verse length: ", average_verse_lengths[-1])
        
        # break ## remove
    print("\nFINAL RESULTS:")
    print("Avg similarity score: ", np.sum(average_sim_scores)/len(average_sim_scores))
    print("Max similarity score: ", max(max_sim_scores))
    print("Avg verse length: ", np.sum(average_verse_lengths)/len(average_verse_lengths))
    
    # break ## remove


    





Testing on  lyrics/kendrick_lamar/GoodKid.txt
unknowns filename: lyrics/kendrick_lamar/GoodKid_unknowns.txt
Need to add 6 words to template word list using LOGIOS tool
Corpus text: 510 words, 252 unique
Iteration:  1
Last avg sim score:  0.015440864504999163
Last max sim score:  0.06557377049180328
Last avg verse length:  120.6


/Users/reedperkins/byu/cs673/generate_sample.py:119: RuntimeWarning: invalid value encountered in true_divide
  scansion_scores /= np.sum(scansion_scores)


	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
Iteration:  21
Last avg sim score:  0.015739664923720965
Last max sim score:  0.042735042735042736
Last avg verse length:  119.0
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
	Already got something here, no need to fetch a word
Iteration:  41
Last avg

/Users/reedperkins/byu/cs673/generate_sample.py:122: RuntimeWarning: invalid value encountered in less
  best_index = np.random.choice(choices,p=scansion_scores)
/Users/reedperkins/byu/cs673/generate_sample.py:305: RuntimeWarning: invalid value encountered in true_divide
  scansion_scores /= np.sum(scansion_scores)
/Users/reedperkins/byu/cs673/generate_sample.py:308: RuntimeWarning: invalid value encountered in less
  best = np.random.choice(len(rhyme_choices), p=scansion_scores)


Iteration:  21
Last avg sim score:  0.01515926446065356
Last max sim score:  0.09090909090909091
Last avg verse length:  99.22
Iteration:  41
Last avg sim score:  0.014787952712797054
Last max sim score:  0.10101010101010101
Last avg verse length:  98.71
Iteration:  61
Last avg sim score:  0.014755471588371438
Last max sim score:  0.09
Last avg verse length:  100.87
Iteration:  81
Last avg sim score:  0.016338410702772407
Last max sim score:  0.11
Last avg verse length:  98.8

FINAL RESULTS:
Avg similarity score:  0.01667651524368583
Max similarity score:  0.20202020202020202
Avg verse length:  99.53059999999999



Testing on  lyrics/drake/NiceForWhat.txt
unknowns filename: lyrics/drake/NiceForWhat_unknowns.txt
Need to add 19 words to template word list using LOGIOS tool
Corpus text: 741 words, 180 unique
Iteration:  1
Last avg sim score:  0.01625548258236474
Last max sim score:  0.08490566037735849
Last avg verse length:  110.93
Iteration:  21
Last avg sim score:  0.017460377029192054

In [153]:
importlib.reload(corpus)
importlib.reload(verse)
importlib.reload(generate_sample)

for idx, song in enumerate(song_files[1:]):
    
    idx += 1
    # print("unknowns: ", unknowns[idx])
    print("\n\n\nTesting on ", song)
    corp = corpus.WordCorpus(dictionary)
    corp.initialize(song)
    corp.add_unknowns(unknowns[idx])
    corp.initializeMatrix()
    corp.initializeSylDict()
    
    template = verse.VerseTemplate(verses[idx], dictionary, breakrules='phrase')
    template.add_unknowns(unknowns[idx])
    print("\nTemplate text:", verses[idx])
    template.get_rhyme()
    result_text = generate_sample.fill_template(corp, template, verbose=False)
    
    
    
    print("\n", result_text)
    
    break

    
    




Testing on  lyrics/cardi-b/BodakYellow.txt
unknowns filename: lyrics/cardi-b/BodakYellow_unknowns.txt
Need to add 13 words to template word list using LOGIOS tool
Corpus text: 768 words, 237 unique

Template text: I'm the hottest in the street, know you prolly heard of me. Got a bag and fixed my teeth, hope you hoes know it ain't cheap. And I pay my mama bills, I ain't got no time to chill. Think these hoes be mad at me, their baby father run a bill. Said, Lil bitch, you can't fuck with me if you wanted to. These expensive, these is red bottoms, these is bloody shoes. Hit the store, I can get 'em both, I don't wanna choose. And I'm quick, cut a nigga off, so don't get comfortable, look.

Fill in Rhyming Words:  #  / bills  /  /  / chill, up months  /  /  #,  #  # think  /  #  /  #,  #  #  # oh  # new  #,  # him  /  #  #  #  #,  / safe  #  #  # who  #,  # steak  #  #  # street  #,  # b  #  #  /  #  /  #,  /, fill fast, bill  # go six  #  # real  # know bitch,  /  #  #  /,  #  /  #  #